# Decision Trees Homework

Goal:
- Decision Tree overfitting

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

### Generate a dataset

In [17]:
# Generate a dataset
X, y = make_classification(
    n_samples=500,
    n_features=50,
    n_informative=5,
    n_redundant=0,
    n_repeated=0,
    n_clusters_per_class=1,
    class_sep=1.2,
    random_state=0
)

### Split the data for training and testing

In [18]:
# Split the data for training and testing

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

### Train a decision tree model

In [24]:
# Train the decision tree model

my_max_depth = None
tree = DecisionTreeClassifier(max_depth=my_max_depth, min_samples_leaf=1, random_state=0)
tree.fit(X_train, y_train)

yhat_train = tree.predict(X_train)
yhat_test = tree.predict(X_test)

train_acc = accuracy_score(y_train, yhat_train)
test_acc = accuracy_score(y_test, yhat_test)
cm = confusion_matrix(y_test, yhat_test)

print(f'training accuracy =  {round(train_acc, 2)}\n')
print(f'test accuracy = {round(test_acc, 2)}\n')
print(f'confusion matrix = \n{cm}')

training accuracy =  1.0

test accuracy = 0.93

confusion matrix = 
[[67  8]
 [ 2 73]]


### (2) The training accuracy = 1.0 whereas the test accuracy is 0.93. This means that the decision tree model has memorized the training data when there is no restriction on the max_depth and min_samples_leaf and therefore the tree is able to acchieve 100% training accuracy. 